<a href="https://colab.research.google.com/github/itayshap/Titanic_Kaggle/blob/main/MLOpsFinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLOPS FINAL PROJECT
###Submmited by:
Baoz 
Omri
Adam
Itay


In [ ]:
# Importing the libraries 
import numpy as np
import pandas as pd
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

### Loading Boston Dataset

In [ ]:
#importing boston dataset
from sklearn.datasets import load_boston
boston = load_boston()
boston_df = pd.DataFrame(boston.data)
boston_df.columns=boston.feature_names
boston_df['PRICE'] = boston.target 
boston_baseline_hyperparams = {}

### Loading French Dataset

In [ ]:
df2 = pd.read_csv("https://www.openml.org/data/get_csv/20649148/freMTPL2freq.arff",
                 quotechar="'")
# rename column names '"name"' => 'name' 
df2.rename(lambda x: x.replace('"', ''), axis='columns', inplace=True)
df2['IDpol'] = df2['IDpol'].astype(np.int64)
french_hyperparams = {'eval_metric':'poisson-nloglik',
                      'objective': 'count:poisson',
                      'colsample_bytree':0.9 ,
                      'learning_rate':0.1,
                      'max_depth':3,
                      'min_child_weight':1,
                      'reg_alpha':5,
                      'subsample':0.9 }

##Creating Generic Pipeline

In [ ]:
class model_pipeline():
  def __init__(self, df, target_column, split_ratio, params):
    self.UNIQUE_VALS_THRESHOLD = 9
    self.model = XGBRegressor() if params == {} else XGBRegressor(params)
    self.df = df
    self.target_column = target_column
    self.split_ratio = split_ratio 
    self.categorical_features, self.numeric_features = self.find_num_cat_features(df)
    self.X_train, self.X_test, self.y_train, self.y_test = self.split_data(df)


  def find_num_cat_features(self,df):
    df_nunique = df.select_dtypes(include=['object']).nunique()
    categorical_features = list(df_nunique[df_nunique < self.UNIQUE_VALS_THRESHOLD].index)
    numeric_features = df.columns.drop(categorical_features).to_list()
    numeric_features.remove(self.target_column)
    return categorical_features, numeric_features

  def split_data(self, df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop([self.target_column], axis = 1), \
                                                        df[self.target_column], test_size = self.split_ratio, random_state = 10)
    return X_train, X_test, y_train, y_test
  
  def create_pipeline(self):  
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder())])

    # RFESelector_transformer = Pipeline(steps=[('RFESelector', RFESelector(10, self.model))])
    RFESelector_transformer = Pipeline(steps=[('RFESelector', RFE(10, self.model))])

    preprocessor = ColumnTransformer(
      transformers=[
        ('numeric', numeric_transformer, self.numeric_features),
        ('all', RFESelector_transformer, self.X_train.columns.to_list()),
        ('categorical', categorical_transformer, self.categorical_features)
    ]) 

    self.pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                ('regressor', self.model)])
    
  def print_metrics(self, target, y_pred):
    print('R^2 =',metrics.r2_score(target, y_pred))
    print('MAE =',metrics.mean_absolute_error(target, y_pred))
    print('MSE =',metrics.mean_squared_error(target, y_pred))
    print('RMSE =',np.sqrt(metrics.mean_squared_error(target, y_pred)))


  def run_pipeline(self):
    self.pipeline.fit(self.X_train, self.y_train)

  def predict(self, mode):
    if mode == "train":
      y_pred = self.pipeline.predict(self.X_train)
      self.print_metrics(self.y_train, y_pred)
    else:
      y_pred = self.pipeline.predict(self.X_test)
      self.print_metrics(self.y_test, y_pred)
      

In [ ]:
model = model_pipeline(boston_df, "PRICE", 0.4, boston_baseline_hyperparams)
model.create_pipeline()
model.run_pipeline()

TypeError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(boston_df.drop("PRICE", axis = 1), boston_df["PRICE"], test_size = 0.4, random_state = 10)

In [ ]:
class RFESelector(BaseEstimator, TransformerMixin):
    def __init__(self, n_features, model):
      self.n_features = n_features
      self.model = model
    #  self.important_features = []
        

    def fit(self, X, y=None):
      selector = RFE(self.model, n_features_to_select=self.n_features)
      selector.fit(X, y)
      self.important_features = selector.support_


    def transform(self, X):
      return X.loc[:, self.important_features]

In [ ]:
class XGBSelector(BaseEstimator, TransformerMixin):
    def __init__(self, n_features, model):
      self.n_features = n_features
      self.model = XGB 
    #  self.important_features = []
        

    def fit(self, X, y=None):
      
      my_dict = w.get_booster().get_score()
      sorted(my_dict, key=my_dict.get, reverse=True)[:10]

      selector = RFE(self.model, n_features_to_select=self.n_features)
      selector.fit(X, y)
      self.important_features = selector.support_


    def transform(self, X):
      return X.loc[:, self.important_features]

In [ ]:
!pip install ydata_synthetic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 2.4 MB/s 
     |████████████████████████████████| 280 kB 8.0 MB/s 
     |████████████████████████████████| 17.1 MB 68.0 MB/s 
     |████████████████████████████████| 11.3 MB 50.7 MB/s 
     |████████████████████████████████| 31.2 MB 1.2 MB/s 
     |████████████████████████████████| 11.7 MB 70.8 MB/s 
     |████████████████████████████████| 511.7 MB 167 bytes/s 
     |████████████████████████████████| 965 kB 47.0 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 33.8 MB 319 kB/s 
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6360 sha256=14e0a85e39ccf9d1b9460c6d4ccfca2d77d95ffeb02734cc54110f21c1ca923f
  Stored in directory: /root/.cache/pip/wheels/d3/e0/e9/305e348717e399665119bd012510d51ff4f22d709ff60c3096
Successfully built easydict
  Attempting uninstall: requests


In [ ]:

from ydata_synthetic.synthesizers.regular import RegularSynthesizer
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters

#Load data and define the data processor parameters
data = fetch_data('adult')
num_cols = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_cols = ['workclass','education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
            'native-country', 'target']

# DRAGAN training
#Defining the training parameters of DRAGAN

noise_dim = 128
dim = 128
batch_size = 500

log_step = 100
epochs = 500+1
learning_rate = 1e-5
beta_1 = 0.5
beta_2 = 0.9
models_dir = '../cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

train_args = TrainParameters(epochs=epochs,
                             sample_interval=log_step)

synth = RegularSynthesizer(modelname='dragan', model_parameters=gan_args, n_discriminator=3)
synth.fit(data = data, train_arguments = train_args, num_cols = num_cols, cat_cols = cat_cols)

synth.save('adult_synth.pkl')

#########################################################
#    Loading and sampling from a trained synthesizer    #
#########################################################
synthesizer = RegularSynthesizer.load('adult_synth.pkl')
synthesizer.sample(1000)

NameError: ignored

Naive implementation of freeAI tool, later it will be added to pipeline

In [ ]:
# Adding to our data a binary column of correct/ incorrect classification
def pre_process_FreeAI(model, X, y, threshold=0.1):
  predictions = model.predict(X)
  correct_classification = pd.Series(np.where(abs(predictions - y) <= threshold * y, True, False))
  return pd.concat((X, correct_classification), axis=1)

In [ ]:
# Testing pre_proccess_freeAI
import xgboost as XGBClassifier
X = boston_df.drop(['PRICE'], axis=1)
y = boston_df['PRICE']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4, random_state = 10)
xgbr = XGBClassifier.XGBRegressor(verbosity=0)
xgbr.fit(X_train, y_train)
new_X = pre_process_FreeAi(xgbr, X_train, y_train, threshold=0.1)
new_X.rename(columns = {0:'accuracy'}, inplace = True)

NameError: ignored

In [ ]:
!pip install dtreeviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 799 kB/s 


In [ ]:
## needs refining for boston data

import plotly.express as px
pd.set_option('display.max_rows', None)

# XGBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# HDP
import scipy.stats.kde as kde
from matplotlib import pyplot as plt

# DT
import graphviz
from dtreeviz.trees import dtreeviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz, _tree
import itertools

#################### HDP ########################
def hpd_grid(sample, alpha=0.05, roundto=2, percent=0.5, show_plot=False):
    """Calculate highest posterior density (HPD) of array for given alpha.
    The HPD is the minimum width Bayesian credible interval (BCI).
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    percent: float
        Perecent of data in the highest density region
    show_plots: bool
        if true, will show intermediary plots
    Returns
    ----------
    hpd: list with the highest density interval
    x: array with grid points where the density was evaluated
    y: array with the density values
    modes: list listing the values of the modes
    Src: https://github.com/aloctavodia/BAP/blob/master/first_edition/code/Chp1/hpd.py
    Note this was modified to find low-accuracy areas
    """

    # data points that create a density plot when histogramed
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    if show_plot: px.histogram(sample, title='Histogram of Bi-Modal Data', template='simple_white', color_discrete_sequence=['#177BCD']).show()

    # get upper and lower bounds on search space
    l = np.min(sample)
    u = np.max(sample)

    # get x-axis values
    x = np.linspace(l, u, 2000)

    # get kernel density estimate
    density = kde.gaussian_kde(sample)
    y = density.evaluate(x)

    if show_plot: px.scatter(x=x, y=y, title='Density Estimate of Bi-Modal Data', template='simple_white').update_traces(marker=dict(color='#177BCD')).show()

    # sort by size of y (density estimate), descending 
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)

    # get all x's where y is in the top 1-alpha percent
    # this is to bound the type 1 error
    xy_cum_sum = 0
    hdv = [] # x values
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break

    # determine horizontal line corresponding to percent 
    yy_zipped = zip(y, y/np.sum(y))
    yy = sorted(yy_zipped, key=lambda x: x[1], reverse=True)

    y_cum_sum = 0
    y_cutoff = 0
    for val in yy:
        y_cum_sum += val[1]
        if y_cum_sum >= percent:
            y_cutoff = val[0]
            break

    # get indices of sample in range 
    intersections = []
    for i, curr in enumerate(y):
        prior = y[i-1]
        if (prior < y_cutoff and curr >= y_cutoff) or (prior >= y_cutoff and curr < y_cutoff):
            intersections.append(x[i])

    indices = []
    for i in range(0, len(intersections), 2):
        lower, upper = intersections[i], intersections[i+1]
        indices.append([i for i,v in enumerate(sample) if v <= upper and v >= lower])

    # setup for difference comparison
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))

    # if y_i - y_{i-1} > diff then save
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))

    # prepare to calcualte value with highest density
    ite = iter(hpd)
    hpd = list(zip(ite, ite)) # create sequential pairs
    modes = []

    # find x and y value whith highest density
    for value in hpd:
         x_hpd = x[(x > value[0]) & (x < value[1])]
         y_hpd = y[(x > value[0]) & (x < value[1])]
         modes.append(round(x_hpd[np.argmax(y_hpd)], roundto)) # store x-value where density is highest in range
    return (hpd, x, y, modes, y_cutoff, np.array(indices).flatten())


############ HDP example ##############

def hpd_example(show_plot=False): 
    """ Plot an example of HDP method on bimodal data.
    Src: https://stackoverflow.com/questions/53671925/highest-density-interval-hdi-for-posterior-distribution-pystan
    """

    # include two modes
    samples = np.random.normal(loc=[-4,4], size=(1000, 2)).flatten()

    # compute high density regions
    hpd_mu, x_mu, y_mu, modes_mu, y_cutoff, indices = hpd_grid(samples, show_plot=False)

    plt.figure(figsize=(8,6))

    # raw data
    plt.hist(samples, density=True, bins=29, alpha=0.5)

    # estimated distribution
    plt.plot(x_mu, y_mu)
    plt.title('Highest Prior Density Region for Bi-Modal Data')

    # high density intervals
    for (x0, x1) in hpd_mu:
        plt.hlines(y=0, xmin=x0, xmax=x1, linewidth=5)
        plt.axvline(x=x0, color='grey', linestyle='--', linewidth=1)
        plt.axvline(x=x1, color='grey', linestyle='--', linewidth=1)

    # modes
    for xm in modes_mu:
        plt.axvline(x=xm, color='r')

    # 95% of data
    plt.axhline(y=y_cutoff, color='g')

    if show_plot: plt.show()

def hpd_iterative_search(col, accuracy, start_percent=0.5, end_percent=0.98, increment=0.05, acc_cutoff=-0.005):
    """
    :param col: (pd.Series) univariate numeric col to search through
    :param accuracy: (pd.Series) boolean accuracy column
    :param start_percent: (flaot) percent to start with
    :param end_percent: (flaot) percent to end with
    :param increment: (float) value to increment by
    :param acc_cutoff: (float) accuracy cutoff to return data
    :return: (2d arry) of indices of problematic areas
    """

    out = [] 
    
    prior_indices = {} 
    prior_acc = None
    prior_p = None 
    percents = np.arange(start_percent, end_percent, increment)[::-1] 

    # get smaller and smaller data slices
    for p in percents:
        # run HDP
        *_, indices = hpd_grid(col, percent=p)

        if indices.shape[0] != 0:

            # get accuracy for HDP
            indices = indices[0] if indices.shape[0] < 10 else indices
            acc = np.mean(accuracy.iloc[indices])

            # determine if there is a "meaningful" change - this is done with a stat sig cal
            if prior_acc is not None and acc - prior_acc < acc_cutoff:
                out.append((f'{col.name}:{p}-{prior_p}', acc - prior_acc, prior_indices - set(indices), acc))

            # reset
            prior_indices = set(indices)
            prior_acc = acc
            prior_p = p

    return out


########## Decision Tree #############

def fit_DT(df, predictors = ['age']):
    """ Fit a classification decision tree and return key elements """

    X = df[predictors] 
    y = df['accuracy_bool'] 

    model = DecisionTreeClassifier(max_depth=3, criterion='entropy', random_state=1)
    model.fit(X, y)

    preds = model.predict(X)
    acc = accuracy_score(y, preds)

    return model, preds, acc, X

def visualize_DT_1(df, model, predictors=['age'], fname='tree'):
    """ Visualize and export DT model """
    data = export_graphviz(model, feature_names=predictors,
                           filled=True, rounded=True, node_ids=True)

    graph = graphviz.Source(data)
    graph.render('trees/'+fname)

def visualize_DT_2(df, model, predictors=['age'], fname='tree'):
    """ Visualize tree with data plots """
    X = df[predictors]
    acc = df['accuracy_bool'].values
    viz = dtreeviz(model, X, acc,
                    target_name="accuracy_bool",
                    feature_names=predictors,
                    class_names=['True','False'] if acc[0] else ['False','True'])

    viz.save('trees/'+fname+'.svg')


def return_dt_split(model, col, accuracy, col_2=None, impurity_cutoff=1.0, n_datapoints_cutoff=5, acc_cutoff=0.03):
    """
    Return all indices of col that meet the following criteria:
    1. Leaf has accuracy lower that baseline by acc_cutoff
    2. Split size > n_datapoints_cutoff 
    :param model: SKLearn classification decision tree model
    :param col: (pd.Series) column used to split on
    :param accuracy: (pd.Series) column corresponding to correct/incorrect classification
    :param col_2: (pd.Series) column to be used for interactions
    :param impurity_cutoff: (float) requirement for entropy/gini of leaf
    :param n_datapoints_cutoff: (int) minimum n in a final node to be returned
    :param acc_cutoff: (float) accuracy cutoff for returning float
    :return: (dict[node_idx, indices]) where indices corresponds to the col that meet the above criteria
    """

    # get leaf ids and setup
    df = pd.concat([col, col_2], axis=1) if col_2 is not None else pd.DataFrame(col)
    leaf_id = model.apply(df)

    t = model.tree_
    baseline_acc = np.mean(accuracy)

    # get indices of leaf ids that meet criteria
    keeps_1 = {i for i,v in enumerate(t.n_node_samples) if v > n_datapoints_cutoff} # sample size
    keeps_2 = {i for i,v in enumerate(t.impurity) if v <= impurity_cutoff} # sample size
    keeps = keeps_1 & keeps_2

    # store all data and corresponding index
    node_indices = {}
    slice_acc = -1
    for idx in keeps:
        node_indices[idx] = [i for i,v in enumerate(leaf_id) if v == idx]

        # remove non-low-accuracy areas and empty lists
        slice_acc = [x[1] / sum(x) for x in t.value[idx]] 
        if baseline_acc - slice_acc < acc_cutoff or node_indices[idx] == []:
            del node_indices[idx]
            slice_acc = None

    return (f'{col.name}{"-"+col_2.name if col_2 is not None else ""}', list(node_indices.keys()), list(node_indices.values()), slice_acc)


###################### Run Helpers #############

def run_data_search(df, viz=2, do_a=True, do_b=True, do_c=True):
    """ Iterate over data columns and perform the following actions...
    1. If type(col) is numeric, run HDP
    2. If type(col) is categorical, run DT
    3. Run DT for interactions
    Save DT viz and print problematic indices
    :param df: (pd.DataFrame) of raw data with correct/incorrect classification
    :param viz: (int) 0 if no viz, 1 if viz type 1, 2 if vis type 2
    """

    categoricals  = [x for x in list(df) if 'color' in x or 'gender' in x]
    acc_col = df['accuracy_bool']

    # store for output
    univariate_numerics_acc = []
    categoricals_acc = []
    bivariate_acc = []

    # univariate loop
    for col_name in [x for x in list(df) if x != 'accuracy_bool']:
        c = df[col_name]

        if col_name not in categoricals and col_name not in ('outcome','accuracy_bool') and do_a:
            print(f'Running: HDP for {col_name}')
            univariate_numerics_acc.append(hpd_iterative_search(c, acc_col))

        # categoricals
        elif do_b:
            print(f'Running: DT for {col_name}')
            predictors = [col_name]
            model, *_, X = fit_DT(df, predictors)

            if viz == 1:
                visualize_DT_1(df, model, predictors=predictors, fname=f'{col_name}')
            elif viz == 2:
                visualize_DT_2(df, model, predictors=predictors, fname=f'{col_name}')

            categoricals_acc.append(return_dt_split(model, c, acc_col))

    # bivariate loop (interactions)
    if do_c:
        for col1, col2 in itertools.combinations(set(df) - set(['accuracy_bool','outcome']), 2):
            print(f'Running: DT for {col1} and {col2}')
            c1, c2 = df[col1], df[col2]
            predictors = [col1,col2]
            model, *_, X = fit_DT(df, predictors=predictors)

            if viz == 1:
                visualize_DT_1(df, model, predictors=predictors, fname=f'{col1}-{col2}')
            elif viz == 2:
                visualize_DT_2(df, model, predictors=predictors, fname=f'{col1}-{col2}')

            bivariate_acc.append(return_dt_split(model, c1, acc_col, c2))

    return (univariate_numerics_acc, categoricals_acc, bivariate_acc)

def clean_output(a, b, c):
    """ 
    Take list of outputs of HPD, DT, and DT interactions and return sorted 
    value by accuracy drop.
    """

    names, indices, accuracies, method = [], [], [], []

    # Univarite using HPD

    # Save vals 
    for v in a:
        for x in v:
            names.append(x[0])
            indices.append(x[2])
            accuracies.append(x[3])
            method.append('HPD')

    for x in b:
        names.append(x[0])
        indices.append(x[2])
        accuracies.append(x[3])
        method.append('DT')

    for x in c:
        names.append(x[0])
        indices.append(x[2])
        accuracies.append(x[3])
        method.append('DT')

    out = pd.DataFrame(dict(names=names, indicies=indices, accuracies=accuracies, method=method))
    out.sort_values(by=['accuracies'], inplace=True)
    out.index = range(len(out.index))

    return out



NameError: ignored

In [ ]:
###################### Run #############

# Step 1: train our baseline model 
df = boston.data
df_test = pre_process_FreeAi(xgbr, X_train, y_train, threshold=0.1) # add preds to df

# Step 2: run HPD Example
#hpd_example(show_plot=False)

# Step 3: run DT example
# Step 3.1: univariate
predictors=['CRIM']
model, preds, acc, X = fit_DT(df_test, predictors=predictors)
visualize_DT_1(df_test, model, predictors=predictors, fname=f'bivariate_demo')
visualize_DT_2(df_test, model, predictors=predictors, fname=f'bivariate_demo')
x = return_dt_split(model, df_test[predictors[0]], df_test['accuracy'])
#print(x)

# Step 3.2: bivariate
predictors = ['glucose','blood_pressure']
model, preds, acc, X = fit_DT(df_test, predictors=predictors)
visualize_DT_1(df_test, model, predictors=predictors, fname=f'bivariate_demo')
visualize_DT_2(df_test, model, predictors=predictors, fname=f'bivariate_demo')
x = return_dt_split(model, df_test[predictors[0]], df_test['accuracy'], df_test[predictors[1]])
#print(x)

# Step 4: find areas of weakness
a, b, c = run_data_search(df_test)
out = clean_output(a,b,c)
out.dropna(inplace=True)
print(out)

NameError: ignored

In [ ]:
# DT
import graphviz
from dtreeviz.trees import dtreeviz
from sklearn.tree import DecisionTreeClassifier, export_graphviz, _tree
import itertools
from sklearn.metrics import accuracy_score


def fit_DT(df):
    """ Fit a classification decision tree and return key elements """

    X = df[:-1] 
    y = df[-1] 

    model = DecisionTreeClassifier(max_depth=3, criterion='entropy', random_state=1)
    model.fit(X, y)

    preds = model.predict(X)
    acc = accuracy_score(y, preds)

    return model, preds, acc, X

def visualize_DT_1(df, model, predictors=['age'], fname='tree'):
    """ Visualize and export DT model """
    data = export_graphviz(model, feature_names=predictors,
                           filled=True, rounded=True, node_ids=True)

    graph = graphviz.Source(data)
    graph.render('trees/'+fname)

def visualize_DT_2(df, model, predictors=['age'], fname='tree'):
    """ Visualize tree with data plots """
    X = df[predictors]
    acc = df['accuracy_bool'].values
    viz = dtreeviz(model, X, acc,
                    target_name="accuracy_bool",
                    feature_names=predictors,
                    class_names=['True','False'] if acc[0] else ['False','True'])

    viz.save('trees/'+fname+'.svg')
